In [29]:
import pandas as pd
import settings
from HelperFunctions import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV



In [9]:
#loading training data
fullTrainData = pd.read_excel(settings.labelledDatapath)
processedTrainData = preprocessDT(fullTrainData)

#splitting features and labels
trainAttributes = processedTrainData.drop(labels=["class"], axis=1)
trainLabels = processedTrainData["class"]


/home/peter/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
classifier = CalibratedClassifierCV(RandomForestClassifier(**settings.optimalHyperparameters), method="isotonic")
classifier.fit(trainAttributes, trainLabels)

CalibratedClassifierCV(estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                                        max_depth=10,
                                                        max_features=None,
                                                        min_samples_split=7,
                                                        n_estimators=90),
                       method='isotonic')

In [14]:
#loading data of potential customers
unseenData = pd.read_excel(settings.unlabelledDatapath)
processedUnseenData = preprocessDT(unseenData, labelled=False)
notInUnseen = list(set(trainAttributes.columns) - set(processedUnseenData.columns))    #adding 0 values for all categorical features seen in training but not in test
processedUnseenData[notInUnseen] = 0
processedUnseenData = processedUnseenData[trainAttributes.columns]    #reordering columns to match train order

#processedUnseenData = processedUnseenData[processedTrainData.columns]

/home/peter/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [16]:
probs = classifier.predict_proba(processedUnseenData)[:,1]
print(probs)

unseenData["prob. >50K"] =  pd.Series(probs)
unseenData.head()

[7.02140309e-04 2.55616209e-01 2.84323622e-01 ... 7.12905328e-01
 7.12644950e-02 7.54366522e-01]


,RowID,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,prob. >50K
0,Row0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0.000702
1,Row1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0.255616
2,Row2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,0.284324
3,Row3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,0.996442
4,Row4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,0.000702


Offers will be sent to potential customers if the probability that they earn >50K is greater than some value, $t$.

$t$ can be defined as the probability threshold of when the expectation of sending a promotion becomes unprofitable (i.e. expected profit > 0 when sending a promotion to a customer whos probability of earning >50K > $t$).

In [17]:
t = 25.5/113.5
#customers to send to are those whose prob. of being >50k is greater than t
unseenData["sendPromotion"] = unseenData["prob. >50K"] > t

unseenData.head()

,RowID,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,prob. >50K,sendPromotion
0,Row0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0.000702,False
1,Row1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0.255616,True
2,Row2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,0.284324,True
3,Row3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,0.996442,True
4,Row4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,0.000702,False


In [33]:
sendPromotion = unseenData[unseenData["prob. >50K"] > t]
sendPromotion["RowID"].to_csv(rowIDsFile, index=False)
sendPromotion.head()

,RowID,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,prob. >50K,sendPromotion
1,Row1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0.255616,True
2,Row2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,0.284324,True
3,Row3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,0.996442,True
7,Row7,63,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,0.607698,True
10,Row10,65,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States,0.977613,True


In [34]:
expectations = sendPromotion["prob. >50K"].map(computeExpectedProfit)
expectations.head()


1      3.512440
2      6.770731
3     87.596128
7     43.473682
10    85.459036
Name: prob. >50K, dtype: float64

In [35]:
estimatedReturns = expectations.sum()
print(estimatedReturns)

227002.5329966998
